This notebook computes ELI5 weights for LightGBM. Currently, there is a number of public kernels showing ELI5 weights for Random Forest Classifiers but I have not seen the LGBM version. I think the latter would be much more relevant for this competition.

If you decide to run this notebook be aware that it takes a few hours to compute the permutation importance (see the step

`perm = PermutationImportance(clf, random_state=RANDOM_STATE).fit(val_x, val_y)` 

below). 

In [1]:
import os
import shutil
import feather
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance

NROUNDS = 1000000
EARLY_STOPPING = 3000
RANDOM_STATE = 44000

###############################################################
# Loading data
###############################################################

print("Loading and spliting data...")

train = feather.read_dataframe('../input/data-serialization/train.feather')
test = feather.read_dataframe('../input/data-serialization/test.feather')
y = feather.read_dataframe('../input/data-serialization/target.feather')
"""
trn_x, val_x, trn_y, val_y = train_test_split(train.values, y.values.ravel(), 
                                              random_state=RANDOM_STATE)
"""
trn_x, val_x, trn_y, val_y = train_test_split(train, y.values.ravel(), random_state=RANDOM_STATE)

Loading and spliting data...


In [2]:
print("Training the classifier...")

clf = lgb.LGBMClassifier(n_estimators=NROUNDS,
                         #**params, 
                         num_leaves=13,
                         boost_from_average=False,
                         objective='binary',
                         max_depth=-1,
                         learning_rate=0.01,
                         boosting='gbdt',
                         min_data_in_leaf=80,
                         bagging_freq=5,
                         bagging_fraction=0.4,
                         feature_fraction=0.05,
                         min_sum_hessian_in_leaf=10.0,
                         tree_learner='serial',
                         #bagging_seed=11,
                         #reg_alpha=5,
                         #reg_lambda=5,
                         metric='auc',
                         verbosity=1,
                         #subsample=0.81,
                         #min_gain_to_split=0.01077313523861969,
                         #min_child_weight=19.428902804238373,
                         num_threads=4)

     
clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=3000,
        early_stopping_rounds=EARLY_STOPPING)

Training the classifier...
Training until validation scores don't improve for 3000 rounds.
[3000]	valid_0's auc: 0.894322
[6000]	valid_0's auc: 0.897899
[9000]	valid_0's auc: 0.898045
Early stopping, best iteration is:
[8403]	valid_0's auc: 0.898164


LGBMClassifier(bagging_fraction=0.4, bagging_freq=5, boost_from_average=False,
        boosting='gbdt', boosting_type='gbdt', class_weight=None,
        colsample_bytree=1.0, feature_fraction=0.05,
        importance_type='split', learning_rate=0.01, max_depth=-1,
        metric='auc', min_child_samples=20, min_child_weight=0.001,
        min_data_in_leaf=80, min_split_gain=0.0,
        min_sum_hessian_in_leaf=10.0, n_estimators=1000000, n_jobs=-1,
        num_leaves=13, num_threads=4, objective='binary',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
        tree_learner='serial', verbosity=1)

In [3]:
print("Computing permtation importances...")

perm = PermutationImportance(clf, random_state=RANDOM_STATE).fit(val_x, val_y)

Computing permtation importances...


In [4]:
print("Computing the ELI5 weights for LightGBM...")

eli5.show_weights(perm, feature_names = val_x.columns.tolist(), top=200)

Computing the ELI5 weights for LightGBM...


Weight,Feature
0.0027 ± 0.0002,var_81
0.0018 ± 0.0004,var_139
0.0016 ± 0.0006,var_26
0.0015 ± 0.0004,var_80
0.0014 ± 0.0003,var_146
0.0014 ± 0.0006,var_198
0.0014 ± 0.0003,var_6
0.0014 ± 0.0003,var_12
0.0014 ± 0.0008,var_76
0.0014 ± 0.0007,var_99
